In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
일반원수계약 = pd.read_excel('./입수파일/일반_원수_계약_20190816.xlsx')
일반원수보상 = pd.read_excel('./입수파일/일반_원수_보상_20190816.xlsx')
일반출재계약 = pd.read_excel('./입수파일/일반_출재_계약_20190812.xlsx')
일반출재보상 = pd.read_excel('./입수파일/일반_출재_보상_20190812.xlsx')
#자동차원수계약 = pd.read_excel('./입수파일/자동차_원수_계약_{}.xlsx'.format(추출년월))
#자동차원수보상 = pd.read_excel('./입수파일/자동차_원수_보상_{}.xlsx'.format(추출년월))
#자동차출재계약 = pd.read_excel('./입수파일/자동차_출재_계약_{}.xlsx'.format(추출년월))
#자동차출재보상 = pd.read_excel('./입수파일/자동차_출재_보상_{}.xlsx'.format(추출년월))

일반특약정보 = pd.read_excel('./입수파일/일반_특약정보_20190811.xlsx')

표준편차평균비율 = pd.read_excel('./입수파일/보유리스크율_기초정보.xlsx', sheet_name='표준편차평균비율', dtype={'보종코드': str, '표준편차평균비율': float})
RMSE준비금비율 = pd.read_excel('./입수파일/보유리스크율_기초정보.xlsx', sheet_name='RMSE준비금비율', dtype={'보종코드': str, 'RMSE준비금비율': float})
RMSE비중 = pd.read_excel('./입수파일/보유리스크율_기초정보.xlsx', sheet_name='RMSE비중')
특약별_정산수수료 = pd.read_excel('./입수파일/특약별_정산수수료.xlsx', dtype={'UY': str})

In [8]:
RMSE비중_가공 = RMSE비중.iloc[:, :9].copy()
RMSE비중_가공 = RMSE비중_가공.melt(id_vars=['구분'], value_vars=RMSE비중_가공.columns[1:]).dropna()
RMSE비중_가공.columns = ['UY', '보종명', 'RMSE비중']
RMSE비중_가공.insert(0, '보종코드', RMSE비중_가공.보종명.apply(lambda x: {'1.화재': 23, '2.기술': 24, '3.종합': 27, '4.해상': 31, '5.근재': 25, '6.책임': 26, '7.상해': 28, '10.일반기타': 29}.get(x)).astype(str))
RMSE비중_가공['보종코드'] = RMSE비중_가공['보종코드'].astype(str)
RMSE비중_가공['UY'] = RMSE비중_가공['UY'].astype(str)

In [9]:
기준년도 = 2018
특약명_전체 = ['재물보험특약', '기술보험_일반', '기술보험_ALOP', '근재보험특약', '배상책임보험특약', '해외PST']
result = []
for 특약명 in 특약명_전체:
    산출기간 = 7 if 특약명 == '근재보험특약' or 특약명 == '배상책임보험특약' else 5
    UY_전체 = np.arange(기준년도-산출기간+1, 기준년도+1).astype(str)
    for UY in UY_전체:
        # 특약코드 추출
        특약코드 = 일반특약정보.loc[일반특약정보['TTY_CD_NM'].str.contains(특약명), 'RRNR_TTY_CD']
        특약코드_UY = 특약코드.loc[특약코드.str.contains('U{}'.format(UY))]

        # 보종코드 추출
        출재계약_특약_UY = 일반출재계약.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        출재보상_특약_UY = 일반출재보상.query('RRNR_TTY_CD in @특약코드_UY').query('UY == @UY')
        보종코드목록 = 출재계약_특약_UY.PDGR_CD.drop_duplicates()

        for 보종코드 in 보종코드목록:
            
            출재계약_특약_UY_보종 = 출재계약_특약_UY.query('PDGR_CD == @보종코드')
            출재보상_특약_UY_보종 = 출재보상_특약_UY.query('PDGR_CD == @보종코드')
        
            # 출재 계약 (보험료)
            출재보험료_UY_국내_원수_보종 = 일반출재계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_PRM'].sum()
            출재보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_PRM'].sum()
            출재보험료비중_특약_UY_보종 = 출재보험료_특약_UY_보종/출재보험료_UY_국내_원수_보종

            # 원수 계약 (보험료)
            원수보험료_UY_국내_원수_보종 = 일반원수계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['WNCR_PRM'].sum()
            원수보험료_특약_UY_보종 = 원수보험료_UY_국내_원수_보종*출재보험료비중_특약_UY_보종
            
            # 출재 계약 (경과보험료)
            출재경과보험료_UY_국내_원수_보종 = 일반출재계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_ELP_PRM'].sum()
            출재경과보험료_특약_UY_보종 = 출재계약_특약_UY_보종['RN_ELP_PRM'].sum()
            출재경과보험료비중_특약_UY_보종 = 출재경과보험료_특약_UY_보종/출재경과보험료_UY_국내_원수_보종
            
            # 원수 계약 (경과보험료)
            원수경과보험료_UY_국내_원수_보종 = 일반원수계약.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['ELP_PRM'].sum()
            원수경과보험료_특약_UY_보종 = 원수경과보험료_UY_국내_원수_보종*출재경과보험료비중_특약_UY_보종

            # 출재 보상 (보험금)
            출재보험금_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OGL_BNF'].sum()
            출재보험금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OGL_BNF'].sum()
            출재보험금비중_특약_UY_보종 = 출재보험금_특약_UY_보종/출재보험금_UY_국내_원수_보종

            # 원수 보상 (보험금)
            원수보험금_UY_국내_원수_보종 = 일반원수보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['OGL_BNF'].sum()
            원수보험금_특약_UY_보종 = 원수보험금_UY_국내_원수_보종*출재보험금비중_특약_UY_보종

            # 출재 보상 (준비금)
            출재준비금_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_OST_AMT'].sum()
            출재준비금_특약_UY_보종 = 출재보상_특약_UY_보종['RN_OST_AMT'].sum()
            출재준비금비중_특약_UY_보종 = 0 if 출재준비금_UY_국내_원수_보종 == 0 else 출재준비금_특약_UY_보종/출재준비금_UY_국내_원수_보종

            # 원수 보상 (준비금)
            원수준비금_UY_국내_원수_보종 = 일반원수보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['OST_AMT'].sum()
            원수준비금_특약_UY_보종 = 원수준비금_UY_국내_원수_보종*출재준비금비중_특약_UY_보종
            
            # 출재 보상 (손해조사비)
#             출재손조비_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['WNCR_CNV_RN_DAG_IVMT'].sum()
#             출재손조비_특약_UY_보종 = 출재보상_특약_UY_보종['WNCR_CNV_RN_DAG_IVMT'].sum()
#             출재손조비비중_특약_UY_보종 = 0 if 출재손조비_UY_국내_원수_보종 == 0 else 출재손조비_특약_UY_보종/출재손조비_UY_국내_원수_보종

            # 원수 보상 (손해조사비)
            #원수손조비_UY_국내_원수_보종 = 일반원수보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['ACCD_IVMT'].sum()
            #원수손조비_특약_UY_보종 = 원수손조비_UY_국내_원수_보종*출재손조비비중_특약_UY_보종
            
            # 출재 보상 (손해액)
            #출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종 + 출재손조비_특약_UY_보종
            출재손해액_특약_UY_보종 = 출재보험금_특약_UY_보종 + 출재준비금_특약_UY_보종
#             출재손해액_UY_국내_원수_보종 = 일반출재보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['RN_LOSS'].sum()
#             출재손해액_특약_UY_보종 = 출재보상_특약_UY_보종['RN_LOSS'].sum()
#             출재손해액비중_특약_UY_보종 = 0 if 출재손해액_UY_국내_원수_보종 == 0 else 출재손해액_특약_UY_보종/출재손해액_UY_국내_원수_보종

#             # 원수 보상 (손해액)
            #원수손해액_특약_UY_보종 = 원수보험금_특약_UY_보종 + 원수준비금_특약_UY_보종 + 원수손조비_특약_UY_보종
            원수손해액_특약_UY_보종 = 원수보험금_특약_UY_보종 + 원수준비금_특약_UY_보종
#             원수손해액_UY_국내_원수_보종 = 일반원수보상.query('UY == @UY').query('NTNL_CTRY_CD == "KR"').query('RRNR_DAT_DVCD == "1"').query('PDGR_CD == @보종코드')['OGL_LOSS'].sum()
#             원수손해액_특약_UY_보종 = 원수손해액_UY_국내_원수_보종*출재손해액비중_특약_UY_보종
            #컬럼 = ['특약명', 'UY', '보종코드', '원수보험료', '출재보험료', '원수경과보험료', '출재경과보험료', '원수보험금', '출재보험금', '원수준비금', '출재준비금', '원수손해조사비', '출재손해조사비', '원수손해액', '출재손해액']
            컬럼 = ['특약명', 'UY', '보종코드', '원수보험료', '출재보험료', '원수경과보험료', '출재경과보험료', '원수보험금', '출재보험금', '원수준비금', '출재준비금', '원수손해액', '출재손해액']
            #데이터 = [특약명, UY, 보종코드, 원수보험료_특약_UY_보종, 출재보험료_특약_UY_보종, 원수경과보험료_특약_UY_보종, 출재경과보험료_특약_UY_보종, 원수보험금_특약_UY_보종, 출재보험금_특약_UY_보종, 원수준비금_특약_UY_보종, 출재준비금_특약_UY_보종, 원수손조비_특약_UY_보종, 출재손조비_특약_UY_보종, 원수손해액_특약_UY_보종, 출재손해액_특약_UY_보종]
            데이터 = [특약명, UY, 보종코드, 원수보험료_특약_UY_보종, 출재보험료_특약_UY_보종, 원수경과보험료_특약_UY_보종, 출재경과보험료_특약_UY_보종, 원수보험금_특약_UY_보종, 출재보험금_특약_UY_보종, 원수준비금_특약_UY_보종, 출재준비금_특약_UY_보종, 원수손해액_특약_UY_보종, 출재손해액_특약_UY_보종]
            df = pd.DataFrame(데이터, 컬럼).T
            result.append(df)
df = pd.concat(result)

In [10]:
df['보종코드'] = df.보종코드.astype(str)
df['원수손해율'] = df['원수손해액']/df['원수경과보험료']
df2 = df.groupby(['특약명', 'UY'])[['원수보험료', '출재보험료', '원수준비금', '출재준비금', '원수보험금', '출재보험금', '원수손해액', '출재손해액']].sum().reset_index()
df2.columns = ['특약명', 'UY', '원수보험료_특약', '출재보험료_특약', '원수준비금_특약', '출재준비금_특약', '원수보험금_특약', '출재보험금_특약', '원수손해액_특약', '출재손해액_특약']
df3 = df.merge(표준편차평균비율, on='보종코드', how='left').drop('보종명', axis=1)
df3['원수손해율'] = df3['원수손해율'].astype(float)
df3['원수손해율변동성'] = df3['원수손해율']*df3['표준편차평균비율']
df3['MU'] = df3.query('원수손해율 > 0').eval('MU = 2*log(원수손해율)-log(원수손해율**2+원수손해율변동성**2)/2')['MU']
df3['MU'] = df3['MU'].fillna(0)
df3['SIGMA'] = df3.eval('SIGMA = sqrt(log(원수손해율**2+원수손해율변동성**2) - 2*log(원수손해율))')['SIGMA']
df3['SIGMA'] = df3['SIGMA'].fillna(0)
df3 = df3.merge(RMSE비중_가공, on=['보종코드', 'UY'], how='left').drop('보종명', axis=1)
df3['원수준비금'] = df3['원수준비금'].astype(float)
temp = df3.groupby(['특약명', '보종코드'])[['원수준비금']].sum().reset_index()
temp.columns = ['특약명', '보종코드', '보종별원수준비금']
df3 = df3.merge(RMSE준비금비율, on='보종코드', how='left').drop('보종명', axis=1)
df3 = df3.merge(temp, on=['특약명', '보종코드'], how='left')
df3.eval('원수준비금RMSE = 보종별원수준비금*RMSE준비금비율*RMSE비중', inplace=True)
df3 = df3.drop(['RMSE비중', 'RMSE준비금비율', '보종별원수준비금'], axis=1)
df3 = df3.merge(특약별_정산수수료, on=['특약명', 'UY'], how='left').drop('정산식', axis=1)
df3 = df3.merge(df2, on=['특약명', 'UY'], how='left')
df3.head()

,특약명,UY,보종코드,원수보험료,출재보험료,원수경과보험료,출재경과보험료,원수보험금,출재보험금,원수준비금,...,Min,Max,원수보험료_특약,출재보험료_특약,원수준비금_특약,출재준비금_특약,원수보험금_특약,출재보험금_특약,원수손해액_특약,출재손해액_특약
0,재물보험특약,2014,23,3.2205e+10,14398997249,3.2205e+10,14398997249,1.35526e+10,5338816732,0.000000e+00,...,0.34,0.6,5.039736e+10,25225008462,1.096073e+07,5800000,1.868924e+10,8504724979,1.870020e+10,8510524979
1,재물보험특약,2014,27,1.81923e+10,10826011213,1.81923e+10,10826011213,5.13665e+09,3165908247,1.096073e+07,...,0.34,0.6,5.039736e+10,25225008462,1.096073e+07,5800000,1.868924e+10,8504724979,1.870020e+10,8510524979
2,재물보험특약,2015,23,4.15515e+10,15224918955,4.15515e+10,15224918955,1.49037e+10,4919202457,3.974627e+09,...,0.35,0.6,6.082548e+10,26574694110,3.998654e+09,910507745,2.549791e+10,10403301987,2.949657e+10,11313809732
3,재물보험특약,2015,27,1.9274e+10,11349775155,1.9274e+10,11349775155,1.05942e+10,5484099530,2.402718e+07,...,0.35,0.6,6.082548e+10,26574694110,3.998654e+09,910507745,2.549791e+10,10403301987,2.949657e+10,11313809732
4,재물보험특약,2016,23,4.30246e+10,11925986697,4.28171e+10,11881681143,1.4495e+10,3551443246,1.480686e+09,...,0.36,0.6,6.681707e+10,24725131030,3.725198e+09,1216161253,3.234087e+10,12214662065,3.606607e+10,13430823318


In [11]:
# 보험가격 보유리스크율
from scipy.stats import norm

result = []
for _, (특약명, UY, 보종코드) in df[['특약명', 'UY', '보종코드']].iterrows():
    data = df3.query('특약명 == @특약명').query('UY == @UY').query('보종코드 == @보종코드').iloc[0]

    mu = data['MU']
    sigma = data['SIGMA']
    원수보험료 = data['원수보험료']
    원수손해액 = data['원수손해액']
    출재보험료 = data['출재보험료']
    출재보험료_특약 = data['출재보험료_특약']
    출재손해액 = data['출재손해액']
    원수손해액_특약 = data['원수손해액_특약']
    출재손해액_특약 = data['출재손해액_특약']
    a = data['a']
    b = data['b']
    기울기 = data['기울기']
    최소 = data['Min']
    최대 = data['Max']

    np.random.seed(20190814)
    난수 = np.random.random(10000)
    명목보유율 = (원수보험료-출재보험료)/원수보험료
    원수손해율_난수 = np.exp(norm(loc=mu, scale=sigma).ppf(난수))
    원수손해액_난수 = 원수손해율_난수*원수보험료
    원수손해액_특약_난수 = 원수손해액_특약-원수손해액+원수손해액_난수
    원수기대손해액 = 원수손해액_특약_난수.mean()
    원수리스크 = np.fmax(원수손해액_특약_난수-원수기대손해액, 0)
    원수기대손실액 = 원수리스크.mean()
    보유손해액_특약_난수 = (원수손해액_특약-출재손해액_특약)-(원수손해액-출재손해액)+원수손해액_난수*명목보유율
    출재손해율_특약_난수 = (원수손해액_특약_난수 - 보유손해액_특약_난수)/출재보험료_특약
    수수료정산액 = 출재보험료_특약*np.fmax(np.fmin(최대, 기울기*(a-출재손해율_특약_난수)+b), 최소)
    보유현금흐름 = 보유손해액_특약_난수 - 수수료정산액
    보유기대손해액 = 보유현금흐름.mean()
    보유리스크 = np.fmax(보유현금흐름-보유기대손해액, 0)
    보유기대손실액 = 보유리스크.mean()
    보유리스크율 = 보유기대손실액/원수기대손실액
    보유율차이 = 보유리스크율 - 명목보유율

    컬럼명 = ['특약명', 'UY', '보종코드', '원수기대손해액', '원수기대손실액', '보유기대손해액', '보유기대손실액', '보유리스크율', '명목보유율', '보유율차이']
    dg = pd.DataFrame([특약명, UY, 보종코드, 원수기대손해액, 원수기대손실액, 보유기대손해액, 보유기대손실액, 보유리스크율, 명목보유율, 보유율차이], 컬럼명).T
    result.append(dg)
dg = pd.concat(result)
dg.head()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1910: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1911: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


,특약명,UY,보종코드,원수기대손해액,원수기대손실액,보유기대손해액,보유기대손실액,보유리스크율,명목보유율,보유율차이
0,재물보험특약,2014,23,1.86854e+10,6.19189e+08,-3.03682e+09,5.63821e+08,0.910579,0.552896,0.357683
0,재물보험특약,2014,27,1.86929e+10,2.86663e+08,-2.86399e+09,2.52545e+08,0.880983,0.404913,0.47607
0,재물보험특약,2015,23,2.94759e+10,8.62512e+08,5.99921e+09,7.9849e+08,0.925772,0.633589,0.292183
0,재물보험특약,2015,27,2.94814e+10,5.91314e+08,6.07349e+09,5.2083e+08,0.8808,0.411135,0.469665
0,재물보험특약,2016,23,3.61259e+10,7.33431e+08,1.31869e+10,5.36658e+08,0.73171,0.72281,0.00889993


In [12]:
# 준비금 보유리스크율
from scipy.stats import norm

result = []
for _, (특약명, UY, 보종코드) in df[['특약명', 'UY', '보종코드']].iterrows():
    data = df3.query('특약명 == @특약명').query('UY == @UY').query('보종코드 == @보종코드').iloc[0]

    원수준비금 = data['원수준비금']
    원수준비금_특약 = data['원수준비금_특약']
    출재준비금 = data['출재준비금']
    출재준비금_특약 = data['출재준비금_특약']
    출재보험료_특약 = data['출재보험료_특약']
    출재보험금_특약 = data['출재보험금_특약']
    출재손해액_특약 = data['출재손해액_특약']   # 출재손해액 정의 다시하기(손조사비 제외하는게 맞는 듯)
    원수준비금RMSE = data['원수준비금RMSE']
    a = data['a']
    b = data['b']
    기울기 = data['기울기']
    최소 = data['Min']
    최대 = data['Max']

    명목보유율 = (원수준비금-출재준비금)/원수준비금 if 원수준비금 != 0 else 0
    출재손해율_특약 = 출재손해액_특약/출재보험료_특약
    수수료정산액 = 출재보험료_특약*np.fmax(np.fmin(최대, 기울기*(a-출재손해율_특약)+b), 최소)

    np.random.seed(20190814)
    난수 = np.random.random(10000)
    원수준비금_난수 = norm(loc=원수준비금, scale=원수준비금RMSE).ppf(난수)
    원수준비금_특약_난수 = 원수준비금_특약 + (원수준비금_난수 - 원수준비금)
    원수기대준비금 = 원수준비금_특약_난수.mean()
    원수리스크 = np.fmax(원수준비금_특약_난수-원수기대준비금, 0)
    원수기대준비금부족액 = 원수리스크.mean()
    보유준비금_특약_난수 = (원수준비금_특약 - 출재준비금_특약) - (원수준비금 - 출재준비금) + 원수준비금_난수*명목보유율
    출재손해율_특약_난수 = (원수준비금_특약_난수-보유준비금_특약_난수+출재보험금_특약)/출재보험료_특약
    수수료차액 = 출재보험료_특약*np.fmax(np.fmin(최대, 기울기*(a-출재손해율_특약_난수)+b), 최소) - 수수료정산액
    
    보유현금흐름 = 보유준비금_특약_난수-수수료차액
    보유기대준비금 = 보유현금흐름.mean()
    보유리스크 = np.fmax(보유현금흐름-보유기대준비금, 0)
    
    보유기대준비금부족액 = 보유리스크.mean()
    보유리스크율 = 보유기대준비금부족액/원수기대준비금부족액
    보유율차이 = 보유리스크율 - 명목보유율

    컬럼명 = ['특약명', 'UY', '보종코드', '원수기대준비금', '원수기대준비금부족액', '보유기대준비금', '보유기대준비금부족액', '보유리스크율', '명목보유율', '보유율차이']
    dh = pd.DataFrame([특약명, UY, 보종코드, 원수기대준비금, 원수기대준비금부족액, 보유기대준비금, 보유기대준비금부족액, 보유리스크율, 명목보유율, 보유율차이], 컬럼명).T
    result.append(dh)
dh = pd.concat(result)
dh.head()

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1910: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1911: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1903: RuntimeWarning: invalid value encountered in greater
  cond0 = self._argcheck(*args) & (scale > 0) & (loc == loc)


,특약명,UY,보종코드,원수기대준비금,원수기대준비금부족액,보유기대준비금,보유기대준비금부족액,보유리스크율,명목보유율,보유율차이
0,재물보험특약,2014,23,8.89416e+06,1.24721e+08,3.50747e+06,9.97768e+07,0.8,0,0.8
0,재물보험특약,2014,27,8.56743e+06,1.44439e+08,3.02072e+06,1.29153e+08,0.894168,0.470838,0.423329
0,재물보험특약,2015,23,3.99587e+09,1.68213e+08,3.08549e+09,1.60608e+08,0.954785,0.773924,0.180861
0,재물보험특약,2015,27,3.99608e+09,1.55151e+08,3.08583e+09,1.3973e+08,0.900604,0.50302,0.397584
0,재물보험특약,2016,23,3.7216e+09,2.17373e+08,2.49368e+09,1.7975e+08,0.826919,0.797554,0.0293657


In [13]:
from datetime import datetime
writer = pd.ExcelWriter('./result/연동수수료특약_UY별_실적집계_{}.xlsx'.format(datetime.now().strftime('%Y%m%d%H%M%S')))
df3.to_excel(writer, index=False)
writer.save()
writer.close()

writer = pd.ExcelWriter('./result/연동수수료특약_표준분포법_보험가격_{}.xlsx'.format(datetime.now().strftime('%Y%m%d%H%M%S')))
dg.to_excel(writer, index=False)
writer.save()
writer.close()

writer = pd.ExcelWriter('./result/연동수수료특약_표준분포법_준비금_{}.xlsx'.format(datetime.now().strftime('%Y%m%d%H%M%S')))
dh.to_excel(writer, index=False)
writer.save()
writer.close()